### 1.6 Definicion de API REST: Inserción de transacciones
En este apartado es el turno de desarrollar una interfaz de tipo API REST para establecer comunicaciones con la _Blockchain_.

In [1]:
import requests
import json

Para probar el endpoint:
- [POST] http://127.0.0.1:8000/new_transaction : Permite insertar nuevas transacciones en la Blockchain. El body de la request es un JSON dict.

Ejemplo de request exitosa:

In [2]:
# URL del endpoint
url = "http://127.0.0.1:8000/new_transaction"

payload = json.dumps({
  "author": "David",
  "content": "Pagos"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Success


Ejemplo de request fallida por datos invalidos:

In [3]:
# URL del endpoint
url = "http://127.0.0.1:8000/new_transaction"

payload = json.dumps({
  "author": "David",
  # No required 'content' field
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Invalid transaction


### 1.7 Definición del API REST: Obtención de la cadena de bloques. 
En esta parte, hemos implementado un _endpoint_ para obtener la cadena de bloques la _Blockchain_.

In [4]:
url = "http://127.0.0.1:8000/chain"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"length": 1, "chain": [{"index": 0, "transactions": [], "timestamp": 1652524479.0852618, "previous_hash": "0", "nonce": 0, "current_hash": "bdb9614babf66c4a47aa084f5ad01529077f5fdb8f1e6eeaf2642c20566db10d"}], "peers": []}


En este caso, como no hemos añadido ningún otro nuevo bloque, observamos que solo contiene el bloque de genesis.

### Blockchain 1.8: Definición del API REST: Minado de transacciones.
En este apartado se ha desarrollado un endpoint _/mine_ de tipo GET, de forma que al lanzar un petición contra esa nueva ruta, la API realiza un mineado de las transacciones pendientes de ser validadas.

In [5]:
url = "http://127.0.0.1:8000/mine"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

Block #1 was mined sucessfully


Si ahora ejecutamos el código del anterior apartado 1.7 vemos como se haincorporado un nuevo bloque a la cadena, que contiene las transacciones que estaban pendientes del punto 1.6.

### 1.9 Definición del API REST: Transacciones sin confirmar.
Desarrollamos una nuevo endpoint para consultar las transacciones pendientes de ser minadas.

Primero generamos nuevas transacciones, de igual forma que antes:

In [6]:
# URL del endpoint
url = "http://127.0.0.1:8000/new_transaction"

payload = json.dumps({
  "author": "Raul",
  "content": "transaction#1"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Success


Ahora lanzamos la request:

In [7]:
url = "http://127.0.0.1:8000/pending_transactions"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

[{'author': 'Raul', 'content': 'transaction#1', 'timestamp': 1652524489.379068}]


Y como se puede comprobar, obtenemos los datos de la transacción enviada anteriormente.

### 1.10: Definición del API REST: Registro de nuevos nodos.
En este aparatado se pretende expandir el sistema de Blockchain, hasta ahora en un único nodo, para implementar una red peer-to-peer.

En primer lugar, vamos a probar a registrar nuevos nodos a la red medinate el endpoint desarrollado _/register_new_node_. 

Un ejemplo de petición fallida (al no tener el campo "new_node_address"):

In [8]:
url = "http://127.0.0.1:8000/register_new_node"

payload = json.dumps({
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Invalid transaction


Añadiendo el campo que faltaba, el resultado es exitoso, y ponemos comprobar como el campo peers contiene la dirección del nodo introducida.

In [24]:
url = "http://127.0.0.1:8000/register_new_node"

payload = json.dumps({
  "new_node_address": "127.0.0.1:8002"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"length": 2, "chain": [{"index": 0, "transactions": [], "timestamp": 1652523941.886769, "previous_hash": "0", "nonce": 0, "current_hash": "6c62b786b6f980d1053f42c600227d02c13a4d87d9bbaed0be94988dd8ae21cc"}, {"index": 1, "transactions": [{"author": "David", "content": "Pagos", "timestamp": 1652523945.2722983}], "timestamp": 1652523946.9714036, "previous_hash": "6c62b786b6f980d1053f42c600227d02c13a4d87d9bbaed0be94988dd8ae21cc", "nonce": 4298, "current_hash": "00086efd67a38c1000bdedea33a1b30367b7984d937f3364e475337c9aa39a91"}], "peers": ["127.0.0.1:8002"]}


En segundo lugar, sometemos a prueba al endpoint desarrollado para registrar un node contra un nodo ya existente de la red _Blockchain_. 

In [9]:
url = "http://127.0.0.1:8000/register_with_existing_node"

payload = json.dumps({
  "node_address": "127.0.0.1:8000"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Self node is already registered.


En este caso nos avisa de que estamos registrando un nodo contra si mismo. Para comprobar un ejemplo exitoso con dos nodos distintos avanzar hasta el apartado 1.13.

### 1.11: Definición del API REST: Definición de Mecanismo de Validación y Consenso.
En esta sección hemos desarrollado un mecanismo de consenso en los nodos, que permite copiar la cadena más larga de la red _Blockchain_ en el nodo.

_Esta funcionalidad (def consensus) se prueba en el apartado 1.13_

### 1.12: Definición del API REST: Propagación de información de minado. 
En este caso hemos implementado un funcionalidad que permite a un nodo difundir en la red un aviso para que los demás nodos actualizen sus cadenas de bloques.

_El endpoint /add_block y la función announce_new_block se prueba en el apartado 1.13_

### 1.13: Definición del API REST: Actualizar ruta de minado.
Por último, en este apartado final se ha sometido a prueba toda la funcionalidad desarrollada. 

Primero generamos una serie de transacciones:

In [10]:
url = "http://127.0.0.1:8000/new_transaction"

payload = json.dumps({
  "author": "Raul",
  "content": "Paga 10 euros a David"
})
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Success


In [11]:
url = "http://127.0.0.1:8000/new_transaction"

payload = json.dumps({
  "author": "David",
  "content": "Devuelve 10 euros a Raul"
})
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Success


Consultamos el estado de las transacciones pendientes. Aparecen las dos enviadas anteriormente.

In [12]:
url = "http://127.0.0.1:8000/pending_transactions"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

[{'author': 'Raul', 'content': 'transaction#1', 'timestamp': 1652524489.379068}, {'author': 'Raul', 'content': 'Paga 10 euros a David', 'timestamp': 1652524503.4788065}, {'author': 'David', 'content': 'Devuelve 10 euros a Raul', 'timestamp': 1652524505.874482}]


Ahora realizamos un minado.

In [13]:
url = "http://127.0.0.1:8000/mine"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

Block #2 was mined sucessfully


Comprobamos que las transacciones se han incorporado a la cadena.

In [14]:
url = "http://127.0.0.1:8000/chain"

payload={}
headers = {}

response_chain_8000 = requests.request("GET", url, headers=headers, data=payload)

print(response_chain_8000.text)

{"length": 3, "chain": [{"index": 0, "transactions": [], "timestamp": 1652524479.0852618, "previous_hash": "0", "nonce": 0, "current_hash": "bdb9614babf66c4a47aa084f5ad01529077f5fdb8f1e6eeaf2642c20566db10d"}, {"index": 1, "transactions": [{"author": "David", "content": "Pagos", "timestamp": 1652524487.543461}], "timestamp": 1652524488.8978558, "previous_hash": "bdb9614babf66c4a47aa084f5ad01529077f5fdb8f1e6eeaf2642c20566db10d", "nonce": 92, "current_hash": "0001cec3d19d1d0f5cd9e2b153d983ee6a8fabe4d53826af475e4ff067440124"}, {"index": 2, "transactions": [{"author": "Raul", "content": "transaction#1", "timestamp": 1652524489.379068}, {"author": "Raul", "content": "Paga 10 euros a David", "timestamp": 1652524503.4788065}, {"author": "David", "content": "Devuelve 10 euros a Raul", "timestamp": 1652524505.874482}], "timestamp": 1652524511.1722827, "previous_hash": "0001cec3d19d1d0f5cd9e2b153d983ee6a8fabe4d53826af475e4ff067440124", "nonce": 233, "current_hash": "000e2edbb68fcb92decd4c8ccda028

**¡IMPORTANTE!** Se debe ejecutar la primera célula del cuaderno **extra_node.ipynb** para levantar un nodo extra de la red.

Con el nuevo nodo levantado, obtenemos su _blockchain_.

In [16]:
url = "http://127.0.0.1:8002/chain"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"length": 1, "chain": [{"index": 0, "transactions": [], "timestamp": 1652524529.4997923, "previous_hash": "0", "nonce": 0, "current_hash": "d9afb0b3f30bbf5d180d4358282fbf5f0f295828e4ad94d8032248076651c5b0"}], "peers": []}


Como era de esperar el nodo únicamente posee el bloque de genesis, por lo que se encuentra desactualizado. Para actualizarlo, lo registramos usando el primer nodo.

In [17]:
url = "http://127.0.0.1:8002/register_with_existing_node"

payload = json.dumps({
  "node_address": "127.0.0.1:8000"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Registration successful


Comprobamos que la cadena se ha actualizado correctamente.

In [18]:
url = "http://127.0.0.1:8002/chain"

payload={}
headers = {}

response_chain_8002 = requests.request("GET", url, headers=headers, data=payload)

print(response_chain_8002.text)

{"length": 3, "chain": [{"index": 0, "transactions": [], "timestamp": 1652524479.0852618, "previous_hash": "0", "nonce": 0, "current_hash": "bdb9614babf66c4a47aa084f5ad01529077f5fdb8f1e6eeaf2642c20566db10d"}, {"index": 1, "transactions": [{"author": "David", "content": "Pagos", "timestamp": 1652524487.543461}], "timestamp": 1652524488.8978558, "previous_hash": "bdb9614babf66c4a47aa084f5ad01529077f5fdb8f1e6eeaf2642c20566db10d", "nonce": 92, "current_hash": "0001cec3d19d1d0f5cd9e2b153d983ee6a8fabe4d53826af475e4ff067440124"}, {"index": 2, "transactions": [{"author": "Raul", "content": "transaction#1", "timestamp": 1652524489.379068}, {"author": "Raul", "content": "Paga 10 euros a David", "timestamp": 1652524503.4788065}, {"author": "David", "content": "Devuelve 10 euros a Raul", "timestamp": 1652524505.874482}], "timestamp": 1652524511.1722827, "previous_hash": "0001cec3d19d1d0f5cd9e2b153d983ee6a8fabe4d53826af475e4ff067440124", "nonce": 233, "current_hash": "000e2edbb68fcb92decd4c8ccda028

Efectivamente la cadena ahora se encuentra actualizada.

In [19]:
print(f"¿Son la misma cadena? {response_chain_8000.json()['chain'] == response_chain_8002.json()['chain']}")

¿Son la misma cadena? True
